In [1]:
from tensorflow.python.client import device_lib
import tensorflow as tf
from skill_label_classifier import *

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/edithzeng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/edithzeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/edithzeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 763477183725581923
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11285289370
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14559069804883178800
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 1d78:00:00.0, compute capability: 3.7"
]
['/job:localhost/replica:0/task:0/device:GPU:0']


In [2]:
experiment = SkillClassifier()
experiment.import_data()
experiment.create_df()

experiment.choose_dataset('vizwiz')
# experiment.set_features(['QSN', 'descriptions', 'tags', 'dominant_colors','handwritten_text', 'ocr_text'])
experiment.set_features(['descriptions'])
experiment.set_targets()

features_train = experiment.features_train
features_val   = experiment.features_val

# check training class distribution
text_recognition_y_train = to_categorical(np.asarray(experiment.txt_train)).astype('float32')
color_recognition_y_train = to_categorical(np.asarray(experiment.col_train)).astype('float32')
print('Number of training samples each class: ')
print('Text recognition', text_recognition_y_train.sum(axis=0))
print('Color recognition', color_recognition_y_train.sum(axis=0))

n_classes = 2

# check validation class distribution
text_recognition_y_val = to_categorical(np.asarray(experiment.txt_val)).astype('float32')
color_recognition_y_val = to_categorical(np.asarray(experiment.col_val)).astype('float32')
print('Number of validation samples each class: ')
print('Text recognition', text_recognition_y_val.sum(axis=0))
print('Color recognition', color_recognition_y_val.sum(axis=0))

# get targets
txt_train      = experiment.txt_train
col_train      = experiment.col_train
txt_val        = experiment.txt_val
col_val        = experiment.col_val

Skipping line 2150: ';' expected after '"'
Skipping line 42: Expected 4 fields in line 42, saw 7
Skipping line 60: Expected 4 fields in line 60, saw 8
Skipping line 157: Expected 4 fields in line 157, saw 5
Skipping line 207: Expected 4 fields in line 207, saw 6
Skipping line 210: Expected 4 fields in line 210, saw 5
Skipping line 275: Expected 4 fields in line 275, saw 5
Skipping line 293: Expected 4 fields in line 293, saw 5
Skipping line 341: Expected 4 fields in line 341, saw 7
Skipping line 357: Expected 4 fields in line 357, saw 5
Skipping line 376: Expected 4 fields in line 376, saw 5
Skipping line 469: Expected 4 fields in line 469, saw 6
Skipping line 478: Expected 4 fields in line 478, saw 5
Skipping line 494: Expected 4 fields in line 494, saw 5
Skipping line 496: Expected 4 fields in line 496, saw 6
Skipping line 504: Expected 4 fields in line 504, saw 5
Skipping line 512: Expected 4 fields in line 512, saw 5
Skipping line 523: Expected 4 fields in line 523, saw 6
Skipping 

Joined features with skill labels.
Joined features with skill labels.
Joined features with skill labels.
Joined features with skill labels.
VizWiz training shape: (14257, 13)
VQA training shape: (3230, 13)
Total training rows: 17487
VizWiz validation shape:(2247, 13)
VQA validation shape:(513, 13)
Total validation rows: 2760
Training: (14257, 13)
Validation: (2247, 13)
Number of training samples each class: 
Text recognition [6247. 8010.]
Color recognition [8844. 5413.]
Number of validation samples each class: 
Text recognition [ 922. 1325.]
Color recognition [1392.  855.]


In [3]:
# tokenize
tok        = Tokenizer(num_words=VOCAB_SIZE, 
                       filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                       lower=True,
                       split=" ")
tok.fit_on_texts(features_train)

# create sequences & pad
train_seq  = tok.texts_to_sequences(features_train)
train_seq  = sequence.pad_sequences(train_seq, maxlen=MAX_DOC_LEN)
val_seq    = tok.texts_to_sequences(features_val)
val_seq    = sequence.pad_sequences(val_seq, maxlen=MAX_DOC_LEN)

# punkt sentence level tokenizer
sent_lst = [] 
for doc in features_train:
    sentences = nltk.tokenize.sent_tokenize(doc)
    for sent in sentences:
        word_lst = [w for w in nltk.tokenize.word_tokenize(sent) if w.isalnum()]
        sent_lst.append(word_lst)

# word2vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
word2vec_model = gensim.models.Word2Vec(sentences=sent_lst,
                                        min_count=6,
                                        size=EMBEDDING_DIM,
                                        sg=1,
                                        workers=os.cpu_count())

embeddings_index = {}
for word in word2vec_model.wv.vocab:
    coefs = np.asarray(word2vec_model.wv[word], dtype='float32')
    embeddings_index[word] = coefs
print('Total %s word vectors' % len(embeddings_index))

# Initial word embedding
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in tok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and i < VOCAB_SIZE:
        embedding_matrix[i] = embedding_vector

2019-03-09 16:41:57,269 : INFO : collecting all words and their counts
2019-03-09 16:41:57,269 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-09 16:41:57,287 : INFO : PROGRESS: at sentence #10000, processed 95361 words, keeping 10420 word types
2019-03-09 16:41:57,291 : INFO : collected 12648 word types from a corpus of 116936 raw words and 12259 sentences
2019-03-09 16:41:57,292 : INFO : Loading a fresh vocabulary
2019-03-09 16:41:57,297 : INFO : effective_min_count=6 retains 302 unique words (2% of original 12648, drops 12346)
2019-03-09 16:41:57,298 : INFO : effective_min_count=6 leaves 104035 word corpus (88% of original 116936, drops 12901)
2019-03-09 16:41:57,299 : INFO : deleting the raw counts dictionary of 12648 items
2019-03-09 16:41:57,300 : INFO : sample=0.001 downsamples 39 most-common words
2019-03-09 16:41:57,300 : INFO : downsampling leaves estimated 32650 word corpus (31.4% of prior 104035)
2019-03-09 16:41:57,301 : INFO : estimated 

Total 302 word vectors


In [4]:
L = 1e-2
R = 0
B = 32
E = 30
val_data = (val_seq, text_recognition_y_val)
start = time.time()
result = lstm_create_train(train_seq, embedding_matrix,
                 train_labels=text_recognition_y_train, skill='text',
                 val_data=val_data,
                 learning_rate=L,
                 lstm_dim=100,
                 batch_size=B,
                 num_epochs=E,
                 optimizer_param=SGD(lr=L, nesterov=True),
                 regularization=R)
end = time.time()
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))
print("----- Total training: {} seconds -----".format(end-start))

Learning rate: 0.01 Regularization: 0 Batch size: 32 Epoch: 30
Accuracy = 0.7578994214508233 	 AUC = 0.8303036876355747
----- Total training: 1442.251312971115 seconds -----


In [5]:
# training accuracy
result.history.keys()

dict_keys(['val_acc', 'val_loss', 'loss', 'acc'])

In [8]:
# 30 epoch
training_accuracy   = result.history['acc']
validation_accuracy = result.history['val_acc']

[0.680017801539655,
 0.7138406764840252,
 0.7254116600441519,
 0.7418780596881216,
 0.7441032488291986,
 0.7449933244856294,
 0.7472185136267064,
 0.7458834001420602,
 0.7476635514283955,
 0.7507788162524296,
 0.7525589675387648,
 0.7521139297370758,
 0.7561192701910143,
 0.7552291945345836,
 0.7565643080192298,
 0.7574543836756605,
 0.7570093458474452,
 0.7561192701644881,
 0.7583444593320914,
 0.7587894971603067,
 0.7592345349885221,
 0.7596795728167376,
 0.7565643079927035,
 0.7592345349885221,
 0.7605696484731683,
 0.7601246106184266,
 0.7623497997860299,
 0.7592345349619959,
 0.7485536271378789,
 0.7659101024117532]

In [ ]:
L = 1e-2
R = 0
B = 32
E = 1000
result = lstm_create_train(train_seq, embedding_matrix,
                 train_labels=text_recognition_y_train, skill='text',
                 val_data=val_data,
                 learning_rate=L,
                 lstm_dim=100,
                 batch_size=B,
                 num_epochs=E,
                 optimizer_param=SGD(lr=L, nesterov=True),
                 regularization=R)
model = load_model('./LSTM/text/{}_{}_{}_{}_model.h5'.format(L,R,B,E))
preds = model.predict(val_seq, verbose=0)
print("Learning rate: {} Regularization: {} Batch size: {} Epoch: {}".format(L,R,B,E))
print(("Accuracy = {0} \t AUC = {1}".format(accuracy_score(txt_val, preds.argmax(axis=1)), 
                                                            roc_auc_score(txt_val, preds[:,1]))))